# Import delle librerie utilizzate e della tabella csv

Inserire il file "Google-Playstore.csv" all'interno della stessa cartella di questo file ed eseguire le celle con i codici.

In [26]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [27]:
path = "Google-Playstore.csv"
df = pd.read_csv(path)
df = df[:100000]
df.head()

print(df.shape)

(100000, 24)


# Pulizia dei dati

La tabella csv originale contiene dati mancanti e/o errati, colonne non utili all'analisi. Si lavora dunque il file per permettere una migliore visualizzazione dei dati.

Verifica dei valori nulli 


In [28]:
print(df.isna().sum().sort_values(ascending = False))

Developer Website    32823
Privacy Policy       18019
Released              2978
Rating                 953
Rating Count           953
Minimum Android        290
Currency                 8
Installs                 7
Minimum Installs         7
Size                     4
Developer Id             1
Developer Email          1
App Name                 1
App Id                   0
Price                    0
Free                     0
Maximum Installs         0
Last Updated             0
Content Rating           0
Category                 0
Ad Supported             0
In App Purchases         0
Editors Choice           0
Scraped Time             0
dtype: int64


Nello specifico, si sostituiscono gli spazi tra le parole che identificano le colonne con un trattino basso.

Vengono successivamente "droppate": 
- le categorie non utili all'analisi
- le righe che contengono un alto numero di valori mancanti (NaN) in OGNI categoria

Si svolge infine un'ulteriore verifica dei valori nulli non più presenti all'interno del dataframe


In [29]:
df.columns = [c.replace(' ', '_') for c in df.columns]

df.drop(columns=['Developer_Website', 'Privacy_Policy', 'Developer_Id', 'Developer_Email', 'Scraped_Time'], inplace = True)
df.dropna(inplace = True)

print(df.isna().sum().sort_values(ascending = False))

App_Name            0
Currency            0
In_App_Purchases    0
Ad_Supported        0
Content_Rating      0
Last_Updated        0
Released            0
Minimum_Android     0
Size                0
Price               0
App_Id              0
Free                0
Maximum_Installs    0
Minimum_Installs    0
Installs            0
Rating_Count        0
Rating              0
Category            0
Editors_Choice      0
dtype: int64


Viene cambiato il formato della colonna relativa alle installazioni ("Installs"), trasformando il tipo di dato da "object" a "int"

In [30]:
df['Installs'] = df['Installs'].str.split('+').str[0]    # rimuove il simbolo '+'
df['Installs'].replace(',','', regex=True, inplace=True) # sostituisce la virgola ',' con un carattere nullo
df['Installs'] = df['Installs'].astype(int)              # conversione da object a int

In [31]:
df['Size'] = df['Size'].apply(lambda x: str(x).replace('M', '') if 'M' in str(x) else x)
df['Size'] = df['Size'].apply(lambda x: str(x).replace(',', '.') if ',' in str(x) else x)
df['Size'] = df['Size'].apply(lambda x: float(str(x).replace('k', '')) / 1000 if 'k' in str(x) else x)
df['Size'] = df['Size'].apply(lambda x: str(x).replace('Varies with device', '0') if 'Varies with device' in str(x) else x)
df['Size'] = df['Size'].apply(lambda x: float(str(x).replace('G', '')) * 1000 if 'G' in str(x) else x)

Conversione delle categorie Rating, Rating Count e Price da stringhe (str) a numeri interi (int)

In [32]:
df['Rating'] = df['Rating'].astype(int)
df['Rating_Count'] = df['Rating_Count'].astype(int)
df['Price'] = df['Price'].astype(int)
df['Minimum_Installs'] = df['Minimum_Installs'].astype(int)
df['Size'] = df['Size'].astype(float)

df['Minimum_Android'].replace(' and up','', regex=True, inplace=True)
df['Size'].replace('M','', regex=True, inplace=True)

Si passa dalla visualizzazione della categoria "Free" alla visualizzazione della categoria "Type". 
Ogni app, a seconda del tipo, può assumere il valore "Free" o "Paid".

In [33]:
df['Type'] = np.where(df['Free'] == True,'Free','Paid')
df.drop(['Free'], axis=1, inplace= True )

Si vanno a creare dei range riguardanti i rating ricevuti dalle singole app. Ogni range è così identificato:
- No Rating Provided
- Less than 10k
- Between 10K and 500K
- More than 500K

In [34]:
df['Rating_Type'] = 'NoRatingProvided'
df.loc[(df['Rating_Count'] > 0) & (df['Rating_Count'] <= 10000.0),'Rating_Type'] = 'Less_than_10K'
df.loc[(df['Rating_Count'] > 10000) & (df['Rating_Count'] <= 500000.0),'Rating_Type'] = 'Between_10K_and_500K'
df.loc[(df['Rating_Count'] > 500000) & (df['Rating_Count'] <= 138557570.0),'Rating_Type'] = 'More_than_500K'
df['Rating_Type'].value_counts()

Less_than_10K           50292
NoRatingProvided        44933
Between_10K_and_500K     1437
More_than_500K             67
Name: Rating_Type, dtype: int64

Si semplifica la classificazione delle apps riguardanti l'età minima dell'utente per scaricarla, creando 3 semplici categorie:
- Everyone
- Teen
- Adults

In [35]:
df["Content_Rating"]=df["Content_Rating"].replace("Unrated","Everyone")
df["Content_Rating"]=df["Content_Rating"].replace("Everyone 10+","Teen")
df["Content_Rating"]=df["Content_Rating"].replace("Adults only 18+","Adults")
df["Content_Rating"]=df["Content_Rating"].replace("Mature 17+","Adults")

Si semplifica anche la visualizzazione dei costi delle apps. 
Si crea una nuova colonna (a partire dalla già esistente "Price") dove vengono inseriti dei range riguardanti il costo delle singole app. Ogni range è così identificato:
- Free
- Low
- Mid
- High

In [36]:
df['Price_Range'] = pd.cut(df['Price'],bins=[0,0.19,9.99,29.99,410],labels=['Free','Low','Mid','High'],include_lowest=True)
df['Price_Range'].value_counts()

Free    95310
Low      1303
Mid        81
High       35
Name: Price_Range, dtype: int64

# Export della tabella csv pulita

Viene creata una nuova tabella csv dove si riportano tutte le modifiche fatte sopra. Il file sarà disponibile all'interno della cartella di lavoro.

In [37]:
df.to_csv("Google-Playstore_clean.csv")

In [38]:
df.head(30)

,App_Name,App_Id,Category,Rating,Rating_Count,Installs,Minimum_Installs,Maximum_Installs,Price,Currency,Size,Minimum_Android,Released,Last_Updated,Content_Rating,Ad_Supported,In_App_Purchases,Editors_Choice,Type,Rating_Type,Price_Range
0,Gakondo,com.ishakwe.gakondo,Adventure,0,0,10,10,15,0,USD,10.0,7.1,"Feb 26, 2020","Feb 26, 2020",Everyone,False,False,False,Free,NoRatingProvided,Free
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4,64,5000,5000,7662,0,USD,2.9,5.0,"May 21, 2020","May 06, 2021",Everyone,True,False,False,Free,Less_than_10K,Free
2,Vibook,com.doantiepvien.crm,Productivity,0,0,50,50,58,0,USD,3.7,4.0.3,"Aug 9, 2019","Aug 19, 2019",Everyone,False,False,False,Free,NoRatingProvided,Free
3,Smart City Trichy Public Service Vehicles 17UC...,cst.stJoseph.ug17ucs548,Communication,5,5,10,10,19,0,USD,1.8,4.0.3,"Sep 10, 2018","Oct 13, 2018",Everyone,True,False,False,Free,Less_than_10K,Free
4,GROW.me,com.horodyski.grower,Tools,0,0,100,100,478,0,USD,6.2,4.1,"Feb 21, 2020","Nov 12, 2018",Everyone,False,False,False,Free,NoRatingProvided,Free
5,IMOCCI,com.imocci,Social,0,0,50,50,89,0,USD,46.0,6.0,"Dec 24, 2018","Dec 20, 2019",Teen,False,True,False,Free,NoRatingProvided,Free
6,unlimited 4G data prank free app,getfreedata.superfatiza.unlimitedjiodataprank,Libraries & Demo,4,12,1000,1000,2567,0,USD,2.5,4.1,"Sep 23, 2019","Sep 27, 2019",Everyone,True,False,False,Free,Less_than_10K,Free
7,The Everyday Calendar,com.mozaix.simoneboard,Lifestyle,2,39,500,500,702,0,USD,16.0,5.0,"Jun 21, 2019","Jun 21, 2019",Everyone,False,False,False,Free,Less_than_10K,Free
9,Neon 3d Iron Tech Keyboard Theme,com.ikeyboard.theme.neon_3d.iron.tech,Personalization,4,820,50000,50000,62433,0,USD,3.5,4.1,"Sep 22, 2019","Oct 07, 2020",Everyone,True,False,False,Free,Less_than_10K,Free
10,Dodge The Cars!,com.MrScratchEnterprises.CarDogeGame,Racing,4,55,100,100,329,0,USD,51.0,4.4,"Jul 30, 2020","Jul 30, 2020",Everyone,False,False,False,Free,Less_than_10K,Free


In [39]:
df.dtypes

App_Name              object
App_Id                object
Category              object
Rating                 int64
Rating_Count           int64
Installs               int64
Minimum_Installs       int64
Maximum_Installs       int64
Price                  int64
Currency              object
Size                 float64
Minimum_Android       object
Released              object
Last_Updated          object
Content_Rating        object
Ad_Supported            bool
In_App_Purchases        bool
Editors_Choice          bool
Type                  object
Rating_Type           object
Price_Range         category
dtype: object